In [ ]:
# clone darknet repo
!git clone https://github.com/AlexeyAB/darknet


In [ ]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile

In [ ]:
# verify CUDA
!/usr/local/cuda/bin/nvcc --version

In [ ]:
# make darknet (builds darknet so that you can then use the darknet executable file to run or train object detectors)
!make

In [ ]:
# define helper functions
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

# use this to upload files
def upload():
  from google.colab import files
  uploaded = files.upload() 
  for name, data in uploaded.items():
    with open(name, 'wb') as f:
      f.write(data)
      print ('saved file', name)

# use this to download a file  
def download(path):
  from google.colab import files
  files.download(path)

In [ ]:
#import google drive to access files easily
%cd ..
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
# this creates a symbolic link so that now the path /content/gdrive/My\ Drive/ is equal to /mydrive
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

In [ ]:
# this is where my zip is stored (I created a yolov3 folder where I will get my required files from)
!ls /mydrive/yolov3

In [ ]:
%cd 

/root


In [ ]:
%cd /content/

/content


In [ ]:
%cd darknet/

/content/darknet


In [ ]:
#copy validation set
!cp /mydrive/yolov3/test.zip ../

In [ ]:
#unzip validationset into data folder
!unzip ../test.zip -d data/

In [ ]:
#copy training set 
!cp /mydrive/yolov3/obj.zip ../

In [ ]:
#unzip training set into data folder
!unzip ../obj.zip -d data/

In [ ]:
# upload the custom .cfg back to cloud VM from Google Drive
!cp /mydrive/yolov3/yolov3_custom3_model2.cfg ./cfg

In [ ]:
# upload the obj.names and obj.data files to cloud VM from Google Drive
!cp /mydrive/yolov3/obj.names ./data
!cp /mydrive/yolov3/obj.data ./data

In [ ]:
!ls data/

In [ ]:
# upload the generate_train.py and generate_test.py script to cloud VM from Google Drive
!cp /mydrive/yolov3/generate_train.py ./
!cp /mydrive/yolov3/generate_test.py ./

In [ ]:
!python generate_train.py

In [ ]:

!python generate_test.py

In [ ]:
# upload pretrained convolutional layer weights
!wget http://pjreddie.com/media/files/darknet53.conv.74

In [ ]:
%cd /content/darknet/

/content/darknet


In [ ]:
#copy, control shift i , console, paste
function ClickConnect(){
console.log("Working"); 
document.querySelector("colab-toolbar-button#connect").click() 
}
setInterval(ClickConnect, 60000)

In [ ]:
# train your custom detector! (uncomment %%capture below if you run into memory issues or your Colab is crashing)
#%%capture

!./darknet detector train data/obj.data cfg/yolov3_custom3.cfg darknet53.conv.74 -dont_show -map

In [ ]:
# kick off training from where it last saved
!./darknet detector train data/obj.data cfg/yolov3_custom3.cfg /mydrive/yolov3/backup/yolov3_custom3_last.weights -dont_show -map

In [ ]:
#code to get any of the saved weights from the training to see the mAP value for that specific weight's file. 

!./darknet detector map data/obj.data cfg/yolov3_custom3_model2.cfg backupopen/yolov3_custom3_5000.weights -thresh 0.02 -iou_thresh 0.40

In [ ]:
#saves all detection prediction results in a result.txt file for specific cfg file and weights. Set iou threshold and confidence threshold
!./darknet detector test data/obj.data cfg/yolov3_custom3_model2.cfg yolov3_custom3_5000.weights -dont_show -thresh 0.02 -iou_thresh 0.40 -ext_output < data/test.txt > result.txt